In [128]:
import spacy
import pandas as pd
from collections import Counter
from spacy.matcher import PhraseMatcher
from spacy.lang.pl import Polish
from spacy.matcher import Matcher
import numpy as np
import re
import PyPDF2

In [129]:
ID_ZESPOLU = "4"

In [130]:
index = ["AEX-", "S&P/ASX 200", "CAC 40", "NASDAQ COMPOSITE", "MSCI COLCAP", "DAX", "DJ US REAL ESTATE", 
"FTSE BURSA MALAYSIA KLCI", "FTSE MIB", "HANG SENG", "IBEX 35", "BRAZIL IBOVESPA," "DOW JONES INDUS. AVG", "S&P/CLX IPSA (CLP) TR",
 "ENERGY SELECT SECTOR", "FINANCIAL SELECT SECTOR", "HEALTH CARE SELECT SECT", "JAKARTA COMPOSITE", "KOSPI", "KOSPI 200", "MSCI EM LAT AM NR",
 "MSCI EUROPE NR", "FTSE 250", "STXE MID 200 (EUR) PR", "MDAX PERF", "S&P MERVAL TR ARS", "S&P/BMV IPC", "WSE MWIG40", "MSCI EAFE", "MSCI EM",
 "MSCI WORLD", "MSCI DAILY TR NET SMALL", "NASDAQ 100 STOCK INDX", "NIFTY 50", "NIKKEI 225", "OMX STOCKHOLM 30", "RTS", "RUSSELL 2000", "CSI 300",
 "SWISS MARKET", "S&P/BVLPERUGENERALTRPEN", "S&P/TSX COMPOSITE", "S&P/TSX 60", "S&P 500", "STRAITS TIMES STI", "WSE SWIG80", "STXE 600 CHEMICALS EUR",
 "EURO STOXX 50 PR", "ESTX BANKS (EUR) PR", "STXE 600 REALESTATE EUR", "STXE 600 HEALTHCARE EUR", "STXE 600 OIL&GAS EUR", "STXE 600 MEDIA (EUR) PR",
 "STXE 600 BASICRESOU EUR", "STXE 600 TRAV&LEISR EUR", "STXE 600 (EUR) PR", "TECDAX PERFORMANCE", "TOPIX (TOKYO)", "TAIWAN TAIEX", "FTSE 100", "VSTOXX",
 "CBOE VOLATILITY", "WSE WIG", "WIG 20", "NASDAQ-100 MICRO", "S&P 500 MINI", "BIST 30", "AEX-", "S&P/ASX200", "CAC40", "NASDAQCOMPOSITE", "MSCICOLCAP", "DAX",
 "DJUSREALESTATE", "FTSEBURSAMALAYSIAKLCI", "FTSEMIB", "HANGSENG", "IBEX35", "BRAZILIBOVESPA," "DOWJONESINDUS.AVG", "S&P/CLXIPSA(CLP)TR", "ENERGYSELECTSECTOR",
 "FINANCIALSELECTSECTOR", "HEALTHCARESELECTSECT", "JAKARTACOMPOSITE", "KOSPI", "KOSPI200", "MSCIEMLATAMNR", "MSCIEUROPENR", "FTSE250", "STXEMID200(EUR)PR",
 "MDAXPERF", "S&PMERVALTRARS", "S&P/BMVIPC", "WSEMWIG40", "MSCIEAFE", "MSCIEM", "MSCIWORLD", "MSCIDAILYTRNETSMALL", "NASDAQ100STOCKINDX", "NIFTY50", "NIKKEI225",
 "OMXSTOCKHOLM30", "RTS", "RUSSELL2000", "CSI300", "SWISSMARKET", "S&P/BVLPERUGENERALTRPEN", "S&P/TSXCOMPOSITE", "S&P/TSX60", "S&P500", "STRAITSTIMESSTI", 
"WSESWIG80", "STXE600CHEMICALSEUR", "EUROSTOXX50PR", "ESTXBANKS(EUR)PR", "STXE600REALESTATEEUR", "STXE600HEALTHCAREEUR", "STXE600OIL&GASEUR", "STXE600MEDIA(EUR)PR",
 "STXE600BASICRESOUEUR", "STXE600TRAV&LEISREUR", "STXE600(EUR)PR", "TECDAXPERFORMANCE", "TOPIX(TOKYO)", "TAIWANTAIEX", "FTSE100", "VSTOXX", "CBOEVOLATILITY",
 "WSEWIG", "WIG20", "NASDAQ-100MICRO", "S&P500MINI", "BIST30","WIBID O/N","MWIG40","SWIG80"]

In [137]:
cols=['ID_KIID',
'ID_ZESPOLU',
'NAZWA_SUBFUNDUSZU',
'NAZWA_FUNDUSZU',
'ISIN',
'IDENTYFIKATOR_KRAJOWY',
'NUMER_RFI',
'PODMIOT_ZARZADZAJACY',
'DATA_AKTUALIZACJI_KIID',
'KATEGORIE_JEDNOSTEK_UCZESTNICTWA',
'CEL_INWESTYCYJNY',
'POLITYKA_INWESTYCYJNA',
'MINIMALNY_POZIOM_INWESTYCJI_UDZIALOWE',
'MAKSYMALNY_POZIOM_INWESTYCJI_UDZIALOWE',
'MINIMALNY_POZIOM_INWESTYCJI_DLUZNE',
'MAKSYMALNY_POZIOM_INWESTYCJI_DLUZNE',
'MINIMALNY_POZIOM_INWESTYCJI_TYTULY_UCZESTNICTWA',
'MAKSYMALNY_POZIOM_INWESTYCJI_TYTULY_UCZESTNICTWA',
'CZESTOTLIWOSC_ZBYWANIA_I_ODKUPOWANIA_JEDNOSTEK_UCZESTNICTWA',
'CZY_FUNDUSZ_WYPLACA_DYWIDENDE',
'BENCHMARK',
'ZALECANY_OKRES_INWESTYCJI',
'PROFIL_RYZYKA_I_ZYSKU',
'SRRI',
'OPLATY',
'OPLATA_ZA_NABYCIE',
'OPLATA_ZA_ODKUPIENIE',
'OPLATY_BIEZACE',
'OPLATY_ZA_WYNIKI',
'WYNIKI_OSIAGNIETE_W_PRZESZLOSCI',
'DATA_PIERWSZEJ_WYCENY',
'DEPOZYTARIUSZ',
'KRS_TOWARZYSTWA',
'NIP_TOWARZYSTWA',
'SIEDZIBA_TOWARZYSTWA',
'KAPITAL_ZAKLADOWY_TOWARZYSTWA',
'WALUTA_KAPITALU_ZAKLADOWEGO_TOWARZYSTWA',
'CZY_ESG',
'TYP_FUNDUSZU']

In [138]:
#NAZWA_SUBFUNDUSZU i NAZWA_FUNDUSZU 
def NAZWA_SUBFUNDUSZU_i_NAZWA_FUNDUSZU(text): 
    result = []
    i = 0
    text = text.replace("\n",".")
    doc = nlp(text)
    for e in doc.ents:
        if e.label_ == "orgName" and "Inwest" in e.text:
            result.append(e.text)
    result = np.array(result)[np.argsort([-len(item) for item in result])][:2].tolist()
    if len(result)==0:
        result = ["NULL","NULL"]
    if len(result)==1:
        result.append("NULL")
    
    return result

In [139]:
#NAZWA_SUBFUNDUSZU i NAZWA_FUNDUSZU 
def NAZWA_SUBFUNDUSZU_i_NAZWA_FUNDUSZU(text): 
    result = []
    i = 0
    text = text.replace("\n",".")
    doc = nlp(text)
    for e in doc.ents:
        if e.label_ == "orgName" and "Inwest" in e.text:
            result.append(e.text)
    result = np.array(result)[np.argsort([-len(item) for item in result])][:2].tolist()
    if len(result)==0:
        result = ["NULL","NULL"]
    if len(result)==1:
        result.append("NULL")
    
    return result

In [140]:
#NAZWA_SUBFUNDUSZU i NAZWA_FUNDUSZU 
def NAZWA_SUBFUNDUSZU_i_NAZWA_FUNDUSZU(text): 
    result = []
    i = 0
    text = text.replace("\n",".")
    doc = nlp(text)
    for e in doc.ents:
        if e.label_ == "orgName" and "Inwest" in e.text:
            result.append(e.text)
    result = np.array(result)[np.argsort([-len(item) for item in result])][:2].tolist()
    if len(result)==0:
        result = ["NULL","NULL"]
    if len(result)==1:
        result.append("NULL")
    
    return result

In [141]:
#ISIN 
def ISIN(text_s):
    tmp=re.findall(r'^(?=.*\d)(?=.*[a-zA-Z])[a-zA-Z\d]{12}$',text_s)
    if len(tmp)>0:
        return tmp[0]
    else:
        return "NULL"

In [142]:
#IDENTYFIKATOR_KRAJOWY 
def IDENTYFIKATOR_KRAJOWY (text_s):
    tmp=re.findall(r'^(?=.*\d)(?=.*[a-zA-Z])[a-zA-Z\d]{11}$',text_s)
    if len(tmp)>0:
        return tmp[0]
    else:
        return "NULL"

In [143]:
#NUMER_RFI
def NUMER_RFI(text):
    rfi=re.findall(r'RFi +\b\d{4}\b',text)
    if(len(rfi)>0):
        rfi=rfi[0].split(" ")
        return rfi[1]
    else:
        return "NULL"

In [144]:
#PODMIOT_ZARZADZAJACY
def PODMIOT_ZARZADZAJACY(text_s):
    text_s = text_s.replace("\n"," ") 
    doc = nlp(text_s)
    for e in doc.ents:
        if "S.A" in e.text:
            return e.text
    return "NULL"
            

In [145]:
#DATA_AKTUALIZACJI_KIID 
def del_multiple_spaces(text):
    if '  ' in text:
        return del_multiple_spaces(text.replace('  ', ' '))
    return text

def clean_str(text_s):
    if '  ' in text_s:
        return del_multiple_spaces(text_s.replace('  ', ' '))
    if text_s[0] == ' ':
        text_s = text_s[1:]
    text_s = text_s.replace(' .', '.')
    text_s = text_s.replace(' ,', ',')
    return text_s

def DATA_AKTUALIZACJI_KIID(text_s):
    text_s = text_s.lower()
    if 'na dzień' in text_s:
        text_s = text_s.split('na dzień')
        if len(text_s) > 1:
            text_s = text_s[1]
        else:
            return "NULL"
        if len(year := re.findall("2 *0 *[0-2] *[0-9]", text_s)) == 0:
            return "NULL"
        text_s = re.split(year[0], text_s)[0] + year[0]
        if len(re.findall("[0-9] [0-9]", text_s)) > 0:
            text_s = text_s.replace(" ", "")
        return clean_str(text_s)
    return "NULL"

In [146]:
#KATEGORIE_JEDNOSTEK_UCZESTNICTWA
def KATEGORIE_JEDNOSTEK_UCZESTNICTWA(text_s):
    tmp=re.findall(r"\s[ABCE] |\s[ABC][1]",text_s)
    if len(tmp)>0:
        return tmp[0].replace(" ","")
    else:
        return "NULL"

In [147]:
#CEL_INWESTYCYJNY 
def CEL_INWESTYCYJNY(text_s):
    result = ""
    text_s = text_s.replace("\n",".")
    doc = nlp(text_s)
    for sent in doc.sents:
        if "cel inwestycyjny" in sent.lemma_.lower():
            result+=sent.text
    return result

In [148]:
#POLITYKA_INWESTYCYJNA
def POLITYKA_INWESTYCYJNA(text_s):
    text_s = text_s.replace("\n"," ")
    doc = nlp(text_s)
    check = False
    for sent in doc.sents:
        if "benchmark" in sent.text.lower() or "dywidend" in sent.text.lower() or "zalecenie" in sent.text.lower():
            check = False
        if check:
            return sent.text
        if "celem inwestycyjn" in sent.text.lower():
            check = True

In [149]:
#MINIMALNY_POZIOM_INWESTYCJI_UDZIALOWE
def MINIMALNY_POZIOM_INWESTYCJI_UDZIALOWE(text_s):
    tmp_min=re.findall(r'od [0-9]+%|co najmniej [0-9]+%|nie więcej niż [0-9]+%', text_s)
    if len(tmp_min)==0:
        return ['NULL','NULL','NULL']
    if len(tmp_min)==1:
        tmp_min=tmp_min[0].split(" ")
        return [tmp_min[-1],'NULL','NULL']
    if len(tmp_min)==2:
        tmp_min_0=tmp_min[0].split(" ")
        tmp_min_1=tmp_min[1].split(" ")
        return [tmp_min_0[-1],tmp_min_1[-1],'NULL']
    if len(tmp_min)>2:
        tmp_min_0=tmp_min[0].split(" ")
        tmp_min_1=tmp_min[1].split(" ")
        tmp_min_2=tmp_min[2].split(" ")
        return [tmp_min_0[-1],tmp_min_1[-1],tmp_min_2[-1]]

In [150]:
#MAKSYMALNY_POZIOM_INWESTYCJI_UDZIALOWE
def MAKSYMALNY_POZIOM_INWESTYCJI_UDZIALOWE(text_s):
    tmp_min=re.findall(r'do [0-9]+%|co najwyżej [0-9]+%|nie mniej niż [0-9]+%', text_s)
    if len(tmp_min)==0:
        return ['NULL','NULL','NULL']
    if len(tmp_min)==1:
        tmp_min=tmp_min[0].split(" ")
        return [tmp_min[-1],'NULL','NULL']
    if len(tmp_min)==2:
        tmp_min_0=tmp_min[0].split(" ")
        tmp_min_1=tmp_min[1].split(" ")
        return [tmp_min_0[-1],tmp_min_1[-1],'NULL']
    if len(tmp_min)>2:
        tmp_min_0=tmp_min[0].split(" ")
        tmp_min_1=tmp_min[1].split(" ")
        tmp_min_2=tmp_min[2].split(" ")
        return [tmp_min_0[-1],tmp_min_1[-1],tmp_min_2[-1]]

In [151]:
#CZESTOTLIWOSC_ZBYWANIA_I_ODKUPOWANIA_JEDNOSTEK_UCZESTNICTWA
def CZESTOTLIWOSC_ZBYWANIA_I_ODKUPOWANIA_JEDNOSTEK_UCZESTNICTWA(text_s):
    text_s = text_s.replace("\n"," ")
    doc = nlp(text_s)
    for token in doc:
        if token.lemma_.lower()=="dzień":
            return "D"
        if token.lemma_.lower()=="tydzień":
            return "W"
        if token.lemma_.lower()=="miesiąc":
            return "M"
        if token.lemma_.lower()=="kwartał":
            return "Q"
    return "NULL"

In [152]:
#CZY_FUNDUSZ_WYPLACA_DYWIDENDE
def CZY_FUNDUSZ_WYPLACA_DYWIDENDE(text_s):
    text_s = text_s.replace("\n"," ")
    doc = nlp(text_s)
    for sent in doc.sents:
        if "dywidenda" in sent.lemma_.lower() and "nie" not in sent.text.lower():
            return True
    return False

In [153]:
#BENCHMARK
def BENCHMARK(text_s, index):
    text_s = text_s.replace("\n"," ")
    doc = nlp(text_s)
    prct = []
    idx = []
    for sent in doc.sents:
        if "benchmark" in sent.text.lower() and "%" in sent.text:
            tmp = sent.text
            prct = re.findall(r'\b[0-9]+%',tmp)
            for item in index:
                if item.lower() in tmp.lower():
                    idx.append(item)
    result = ""
    for i in range(min(len(prct),len(idx))):
        if i == len(prct)-1:
            result+=(prct[i]+" ")
            result+=(idx[i])
        else:
            result+=(prct[i]+" ")
            result+=(idx[i])
            result+=" + "
    return result

In [154]:
#ZALECANY_OKRES_INWESTYCJI
def ZALECANY_OKRES_INWESTYCJI(text_s):
    if 'może nie być odpowiedni' in text_s:
        text_s = text_s.split('może nie być odpowiedni')
        if len(text_s) <= 1:
            return ''
        text_s = text_s[1]
        text_s = re.split("roku|lat", text_s)[0]
        for token in text_s.split(' '):
            if token.isnumeric():
                return int(token)
    else:
        return "NULL"

In [155]:
#PROFIL_RYZYKA_I_ZYSKU
def PROFIL_RYZYKA_I_ZYSKU(text_s):
    result = ""
    text_s = text_s.replace("\n"," ")
    doc = nlp(text_s)
    check = False
    for sent in doc.sents:
        if "profil ryzyka i zysku" in sent.text.lower():
            check = True
        if "opłata" in sent.lemma_.lower():
            check = False
        if check:
            result+=sent.text

In [156]:
#SRRI
def SRRI(text_s):
    text_s = text_s.replace("\n"," ")
    doc = nlp(text_s)
    for token in doc:
        if token.lemma_.lower() == "kategoria" and token.right_edge.like_num:
            return token.right_edge.text

In [157]:
#OPLATY
def OPLATY(text_s):
    text_s = text_s.replace("\n"," ")
    doc = nlp(text_s)
    check = False
    result = ""
    for sent in doc.sents:
        if "opłata" in sent.lemma_.lower() and 'opłata za nabycie' not in sent.text.lower() and 'opłata za odkupienie' not in sent.text.lower() and 'opłaty bieżące' not in sent.text.lower() and 'opłaty za wyniki' not in sent.text.lower():
            check = True
        if "wyniki osiągnięte w przeszłości" in sent.text.lower():
            if len(result)==0:
                result="NULL"
            return result
        if check:
            result+=sent.text
    if len(result)==0:
        result="NULL"
    return result

In [158]:
#OPLATY_ROZNE 
def OPLATY_ROZNE(text_s):
    tmp=['Opłata za nabycie','Opłata za odkupienie','Opłaty bieżące','Opłaty za wyniki']
    tab=[]
    for t in tmp:
        zzz=re.findall(fr'{t}[\D]+[0-9],[0-9][0-9]%',text_s.replace('\n'," "))
        if(len(zzz)>0):
            tab.append(zzz[0].split(" ")[-1])
        else:
            tab.append("NULL")
    return tab

In [159]:
#WYNIKI_OSIAGNIETE_W_PRZESZLOSCI
def split_ignoring_spaces(delimiter, text, ignore_case=True):
    return re.split(" *".join([char for char in delimiter]), text, flags=re.IGNORECASE if ignore_case else 0)

def WYNIKI_OSIAGNIETE_W_PRZESZLOSCI(text_s): 
    text_s = text_s.replace("\n", " ")
    splitted = split_ignoring_spaces('Wyniki osiągnięte w przeszłości', text_s)
    return clean_str(
        split_ignoring_spaces('Niniejsze kluczowe',
            split_ignoring_spaces('Informacje praktyczne',
                splitted[1] if len(splitted) > 1 else splitted[0]
            )[0]
        )[0]
    )

In [160]:
#STOPY!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

In [161]:
#DATA_PIERWSZEJ_WYCENY 
def DATA_PIERWSZEJ_WYCENY(text_s):
    
    text_s = text_s.lower()
    delim = None
    
    for c in ('data pierwszej wyceny', 'pierwsza wycena'):
        if c.lower() in text_s:
            delim = c
            break
    
    text_s = text_s.split(delim)
    if type(text_s) != list: return 'NULL'
    if len(text_s) <= 1: return 'NULL'
    
    text_s = text_s[1]
    
    numbers = re.findall("[0-9]", text_s)
    if len(numbers) == 0: return 'NULL'
    
    text_s = text_s.split(numbers[0])
    if type(text_s) != list: return 'NULL'
    if len(text_s) <= 1: return 'NULL'

    text_s = numbers[0] + ''.join(text_s[1:])
        
    if len(year := re.findall("2 *0 *[0-2] *[0-9]", text_s)) == 0:
        return "NULL"
        
    text_s = re.split(year[0], text_s)[0] + year[0]

    if len(re.findall("[0-9] [0-9]", text_s)) > 0:
        text_s = text_s.replace(" ", "")
    return clean_str(text_s)

In [162]:
#DEPOZYTARIUSZ
def DEPOZYTARIUSZ(text_s):
    text_s = text_s.replace("\n"," ")
    doc = nlp(text_s)
    for sent in doc.sents:
        if "depozytariusz" in sent.lemma_.lower():
            for e in sent.ents:
                if "S.A" in e.text or "Spółka" in e.text:
                    return e.text

In [163]:
#KRS_TOWARZYSTWA 
def KRS_TOWARZYSTWA(text_s):
    krs=re.findall(r'KRS +\b\d{10}\b',text_s)
    if(len(krs)>0):
        krs=krs[0].split(" ")
        return krs[1]
    else:
        return "NULL"

In [164]:
#NIP_TOWARZYSTWA
def NIP_TOWARZYSTWA(text_s):
    nip=re.findall(r'\b\d{3}-\d{2}-\d{2}-\d{3}\b',text_s)
    if len(nip)==0:
        return "NULL"
    return nip[0]

In [165]:
#SZIEDZIBA_TOWARZYSTWA
def SIEDZIBA_TOWARZYSTWA(text_s):
    isset = False
    for c in ('ul.', 'al.', 'ulica', 'aleja', 'Aleje', 'Plac'):
        isset = isset | (c.lower() in text_s.lower())
        if isset:
            delim = c
            break
    if not isset:
        return ''
    text_s = text_s.split(delim)
    text_s = text_s[1] if len(text_s) > 1 else text_s[0]
    for c in ('Warszawa', 'Warszawie', 'Poznań', 'Poznaniu', 'Wrocław', 'Wrocławiu', 'Kraków', 'Krakowie', 'Gdynia', 'Gdyni', 'Sopot', 'Sopocie'):
        if c in text_s:
            text_s = text_s.split(c)
            end = c
            text_s = text_s[0]
            break
    return clean_str(delim + text_s + end)

In [166]:
#KAPITAL_ZAKLADOWY_TOWARZYSTWA
def KAPITAL_I_WALUTA_ZAKLADOWA_TOWARZYSTWA(text_s):
    tmp=re.findall(r"\b[0-9][0-9|\s]{4,},[0-9][0-9]",text_s)
    if(len(tmp)>0):
        kapital=tmp[0]
    else:
        return ['NULL','NULL']
    waluta=re.findall(fr"{kapital}[\s][A-Za-zł]+\b",text_s)
    if(len(waluta)>0):
        return [kapital,waluta[0].split()[-1]]
    else:
        return [kapital,'NULL']

In [167]:
#CZY_ESG
def CZY_ESG(text_s):
    text_s = text_s.replace("\n"," ")
    doc = nlp(text_s)
    for token in doc:
        if token.text.lower() == "esg":
            return True
    return False

In [168]:
#TYP_FUNDUSZU
def TYP_FUNDUSZU(text_s):
    tmp=re.findall(r'FIO',text_s)
    tmp2=re.findall(r'SFIO',text_s)
    if(len(tmp)>0):
        return "FIO"
    elif(len(tmp2)>0):
        return "SFIO"
    else:
        return "NULL"

In [169]:
def pdf_2_text(pdf):
    return_str = False
    if type(pdf) == str:
        pdf = [pdf]
        return_str = True
    
    content = dict()
    
    for fname in pdf:
        
        print(f"Reading {fname}...")
        
        content[fname] = ''
        
        pdfFileObj = open("data/" + fname, 'rb') 
        pdfReader = PyPDF2.PdfFileReader(pdfFileObj) 
        
        try:
            numPages = pdfReader.numPages
        except:
            pass
        else:
            for j in range(pdfReader.numPages):
                pageObj = pdfReader.getPage(j)
                content[fname] += pageObj.extractText().replace("\n", ' ')

        pdfFileObj.close()
            
    return list(content.values())[0] if return_str else content

In [171]:
def MakeOmegaTable(df_meta):
    pdfs = df_meta["NAZWA_PLIKU"].values
    id_kiid = df_meta["ID_KIID"].values
    df_omega=pd.DataFrame(columns=cols)
    
    for i in range(len(pdfs)):
        text_s = pdf_2_text(pdfs[i])
        
        vector = [id_kiid[i],
                  ID_ZESPOLU,
                  *NAZWA_SUBFUNDUSZU_i_NAZWA_FUNDUSZU(text_s),
                  ISIN(text_s),
                  IDENTYFIKATOR_KRAJOWY(text_s),
                  NUMER_RFI(text_s),
                  PODMIOT_ZARZADZAJACY(text_s),
                  DATA_AKTUALIZACJI_KIID(text_s),
                  KATEGORIE_JEDNOSTEK_UCZESTNICTWA(text_s),
                  CEL_INWESTYCYJNY(text_s),
                  POLITYKA_INWESTYCYJNA(text_s),
                  MINIMALNY_POZIOM_INWESTYCJI_UDZIALOWE(text_s)[0],
                  MAKSYMALNY_POZIOM_INWESTYCJI_UDZIALOWE(text_s)[0],
                  MINIMALNY_POZIOM_INWESTYCJI_UDZIALOWE(text_s)[1],
                  MAKSYMALNY_POZIOM_INWESTYCJI_UDZIALOWE(text_s)[1],
                  MINIMALNY_POZIOM_INWESTYCJI_UDZIALOWE(text_s)[2],
                  MAKSYMALNY_POZIOM_INWESTYCJI_UDZIALOWE(text_s)[2],
                  CZESTOTLIWOSC_ZBYWANIA_I_ODKUPOWANIA_JEDNOSTEK_UCZESTNICTWA(text_s),
                  CZY_FUNDUSZ_WYPLACA_DYWIDENDE(text_s),
                  BENCHMARK(text_s,index),
                  ZALECANY_OKRES_INWESTYCJI(text_s),
                  PROFIL_RYZYKA_I_ZYSKU(text_s),
                  SRRI(text_s),
                  OPLATY(text_s),
                  *OPLATY_ROZNE(text_s),
                  WYNIKI_OSIAGNIETE_W_PRZESZLOSCI(text_s),
                  DATA_PIERWSZEJ_WYCENY(text_s),
                  DEPOZYTARIUSZ(text_s),
                  KRS_TOWARZYSTWA(text_s),
                  NIP_TOWARZYSTWA(text_s),
                  SIEDZIBA_TOWARZYSTWA(text_s),
                  *KAPITAL_I_WALUTA_ZAKLADOWA_TOWARZYSTWA(text_s),
                  CZY_ESG(text_s),
                  TYP_FUNDUSZU(text_s)]
        
        
        print(len(vector))
        
        df2 = pd.DataFrame([vector], columns=cols)
        df_omega = pd.concat([df_omega, df2]).reset_index(drop=True)
    df_omega.to_csv("TMP_KIID_DANE.csv",index=False,encoding="cp1250",sep=";")

In [172]:
df_meta = pd.read_csv("TMP_KIID_META.csv",encoding="cp1250",sep=";")
nlp = spacy.load("pl_core_news_md")

In [ ]:
MakeOmegaTable(df_meta)